In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

/home/vipuser/anaconda3/envs/lr-sql/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_path = '/home/model/para_Qwen' 
lora_path = './Qwen2_SFT_Table/final_checkpoint'
# val_dataset_path = "/home/code/chat_SQL/Dataset/GLM4/ENG_ori/My_idea_dataset/T2Q_GLM4_SFT_val_table.jsonl"

In [4]:
# model = AutoPeftModelForCausalLM.from_pretrained(
#         lora_path, attn_implementation="flash_attention_2",torch_dtype=torch.bfloat16,trust_remote_code= True, device_map='auto')
model = AutoPeftModelForCausalLM.from_pretrained(
        lora_path,torch_dtype=torch.bfloat16,trust_remote_code= True, device_map='auto')

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 4/4 [00:39<00:00,  9.96s/it]


In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# 拆解训练集
# 提供每次训练token数
from tqdm import tqdm
import copy 
import re
max_token = 1100
#  保证特殊字符能被识别
accept_toekn = max_token - 170
#获取REF表数据
tab_str = ""
schema_slicing = []
df = pd.read_csv("./dataset2/table_schema_Reference_cropped.csv",encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['Reference_group']
    tab_str_elder = copy.deepcopy(tab_str)
    tab_str += table_group + "\n"
    check_token = tokenizer(f"{tab_str}", add_special_tokens=False)
    if len(check_token["input_ids"]) > accept_toekn:
        schema_slicing.append(tab_str_elder)
        tab_str = table_group + "\n"

    
print(len(schema_slicing))

# print(ReF_group[5])
# print("#############################")
# print(ReF_group[6])


#获取noREF表数据
df = pd.read_csv("./dataset2/table_schema_noReference_cropped.csv",encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['noReference_group']
    tab_str_elder = copy.deepcopy(tab_str)
    tab_str += table_group + "\n"
    check_token = tokenizer(f"{tab_str}", add_special_tokens=False)
    if len(check_token["input_ids"]) > accept_toekn:
#         print("+++++++++++")
        schema_slicing.append(tab_str_elder)
        tab_str = table_group + "\n"
        
        
if len(tab_str) > 0:
    schema_slicing.append(tab_str)

print(len(schema_slicing))
# print(schema_slicing[6])
# print("#############################")
# print(schema_slicing[7])
# print(schema_slicing[8])

create_table_pattern = re.compile(r'CREATE TABLE `[^`]+` \([^;]+\);')
table_name_group = []
test_total = set()
count = 0
for piece in schema_slicing:
    create_table_statements = create_table_pattern.findall(piece)
    temp_list = []
    for table in create_table_statements:
        table_name = re.search(r"CREATE TABLE `([^`]+)`", table).group(1)
        temp_list.append(table_name.lower())
        test_total.add(table_name.lower())
        count+=1
    print(temp_list)
    table_name_group.append(temp_list)
# print(count)
print(len(test_total))
#     table_name_group.append()

100%|███████████████████████████████████████████████████████████████████████████████| 142/142 [00:00<00:00, 1155.49it/s]


6


100%|█████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 1037.12it/s]

8
['manufacturers', 'products', 'student', 'plays_games', 'sportsinfo', 'actor', 'entrepreneur', 'people', 'ref_hotel_star_ratings', 'ref_attraction_types', 'hotels', 'tourist_attractions', 'street_markets', 'shops', 'museums', 'royal_family', 'theme_parks', 'visits', 'photos', 'staff']
['tourist_attraction_features', 'wrestler', 'elimination', 'business', 'category', 'checkin', 'neighbourhood', 'review', 'tip', 'ref_detention_type', 'ref_incident_type', 'addresses', 'students', 'teachers', 'assessment_notes', 'behavior_incident', 'detention']
['student_addresses', 'students_in_detention', 'film', 'film_market_estimation', 'catalogs', 'catalog_structure', 'catalog_contents', 'catalog_contents_additional_attributes', 'routes', 'airports', 'stadium', 'game', 'injury_accident', 'physician', 'department', 'affiliated_with', 'trained_in', 'patient', 'nurse']
['appointment', 'prescribes', 'block', 'room', 'on_call', 'stay', 'undergoes', 'buildings', 'office_locations', 'region', 'party', 'me

In [9]:
# create_random_slice   
import random
w = 8
create_table_pattern = re.compile(r'CREATE TABLE `[^`]+` \([^;]+\);')
table_name_group = []
test_total = set()
count = 0
total_table_list = []
for piece in schema_slicing:
    create_table_statements = create_table_pattern.findall(piece)
    for table in create_table_statements:
        total_table_list.append(table)
print(len(total_table_list))
print(total_table_list)

172
['CREATE TABLE `Manufacturers` (\r\n  Code INTEGER,\r\n  Name VARCHAR(255),\r\n  Headquarter VARCHAR(255),\r\n  Founder VARCHAR(255),\r\n  Revenue REAL\r\n);', 'CREATE TABLE `Products` (\r\n  Code INTEGER,\r\n  Name VARCHAR(255),\r\n  Price DECIMAL,\r\n  Manufacturer INTEGER REFERENCES Manufacturers(Code)\r\n);', 'CREATE TABLE `Student` (\r\n  StuID INTEGER,\r\n  LName VARCHAR(12),\r\n  Fname VARCHAR(12),\r\n  Age INTEGER,\r\n  Sex VARCHAR(1),\r\n  Major INTEGER,\r\n  Advisor INTEGER,\r\n  city_code VARCHAR(3)\r\n);', 'CREATE TABLE `Plays_Games` (\r\n  StuID INTEGER REFERENCES Student(StuID),\r\n  GameID INTEGER REFERENCES Video_Games(GameID),\r\n  Hours_Played INTEGER\r\n);', 'CREATE TABLE `SportsInfo` (\r\n  StuID INTEGER REFERENCES Student(StuID),\r\n  SportName VARCHAR(32),\r\n  HoursPerWeek INTEGER,\r\n  GamesPlayed INTEGER,\r\n  OnScholarship VARCHAR(1)\r\n);', 'CREATE TABLE `actor` (\r\n  Actor_ID INT PRIMARY KEY,\r\n  Name TEXT,\r\n  Musical_ID INT REFERENCES actor(Actor_ID

In [10]:
def shuffle_list(lst, seed=42):
    random.seed(seed)              # 固定随机种子
    shuffled = lst[:]              # 复制一份，避免修改原列表
    random.shuffle(shuffled)       
    return shuffled
total_table_list = shuffle_list(total_table_list)
print(total_table_list)

['CREATE TABLE `Ref_Incident_Type` (\r\n  incident_type_code VARCHAR(10) PRIMARY KEY,\r\n  incident_type_description VARCHAR(80)\r\n);', 'CREATE TABLE `Stay` (\r\n  StayID INTEGER,\r\n  Patient INTEGER REFERENCES Patient(SSN),\r\n  Room INTEGER REFERENCES Room(RoomNumber),\r\n  StayStart DATETIME,\r\n  StayEnd DATETIME\r\n);', 'CREATE TABLE `Medication` (\r\n  Code INTEGER,\r\n  Name VARCHAR(30),\r\n  Brand VARCHAR(30),\r\n  Description VARCHAR(30)\r\n);', 'CREATE TABLE `CMI_Cross_References` (\r\n  cmi_cross_ref_id INTEGER,\r\n  master_customer_id INTEGER REFERENCES Customer_Master_Index(master_customer_id),\r\n  source_system_code CHAR(15)\r\n);', 'CREATE TABLE `country` (\r\n  Country_ID INT PRIMARY KEY,\r\n  Name TEXT,\r\n  Population INT,\r\n  Area INT,\r\n  Languages TEXT\r\n);', 'CREATE TABLE `Affiliated_With` (\r\n  Physician INTEGER PRIMARY KEY REFERENCES Physician(EmployeeID),\r\n  Department INTEGER PRIMARY KEY REFERENCES Department(DepartmentID),\r\n  PrimaryAffiliation BOO

In [11]:
def split_list(lst, parts=8):
    """把列表均分成 parts 份，返回字符串列表，每个子列表用换行符连接"""
    n = len(lst)
    avg = n // parts
    remainder = n % parts
    
    result = []
    start = 0
    for i in range(parts):
        end = start + avg + (1 if i < remainder else 0)
        # 把子列表转成字符串，用 \n 连接
        chunk_str = "\n".join(map(str, lst[start:end]))
        result.append(chunk_str)
        start = end
    return result
schema_slicing = split_list(total_table_list, parts=8)

# for i, sub in enumerate(schema_slicing, 1):
#     print(f"子列表{i}: {sub}")


create_table_pattern = re.compile(r'CREATE TABLE `[^`]+` \([^;]+\);')
table_name_group = []
test_total = set()
count = 0
for piece in schema_slicing:
#     print(piece)
    create_table_statements = create_table_pattern.findall(piece)
    temp_list = []
    for table in create_table_statements:
        table_name = re.search(r"CREATE TABLE `([^`]+)`", table).group(1)
        temp_list.append(table_name.lower())
        test_total.add(table_name.lower())
        count+=1
    print(temp_list)
    table_name_group.append(temp_list)
# print(count)
print(len(test_total))

['ref_incident_type', 'stay', 'medication', 'cmi_cross_references', 'country', 'affiliated_with', 'ref_address_types', 'student', 'furniture', 'office_locations', 'station_company', 'publication', 'companies', 'visitors', 'region', 'staff', 'bridge', 'tweets', 'party', 'enzyme', 'shops', 'catalog_contents']
['store', 'university', 'institution', 'catalog_structure', 'injury_accident', 'detention', 'visits', 'store_district', 'events', 'catalog_contents_additional_attributes', 'manufacturers', 'product', 'services', 'transactions_lots', 'trained_in', 'parking_fines', 'purchases', 'students_in_detention', 'patient', 'procedures', 'web_client_accelerator', 'participants_in_events']
['benefits_overpayments', 'plays_games', 'store_product', 'train', 'students', 'district', 'museums', 'mill', 'checkin', 'attribute_definitions', 'journalist', 'broadcast', 'on_call', 'theme_parks', 'lots', 'apartments', 'person', 'royal_family', 'gas_station', 'teachers', 'photos', 'apartment_buildings']
['pap

In [13]:
def predict(messages, model, tokenizer):
    device = "cuda"
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        pad_token_id=tokenizer.eos_token_id 
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
#     print(response)
     
    return response

# instruction = row['instruction']
# input_value = row['input']
def get_instruction(schema):
    instruction = f" I want you to act as a relation extraction robot for a sample SQL table. You only need to return the tables related to the user's input question. Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. \n##instruction:{schema}"
    return instruction

def get_input_value(question, selected_table):
    input_value = f"{question}######selected_table: {selected_table}."
    return input_value

In [14]:
df = pd.read_csv("./dataset2/validation_dataset_formatted_cropped.csv",encoding="utf-8")
outer_index = 0
import re
import time
pattern = re.compile(r'-- Tables: ([\w, ]+);')
results = []
#  schema_slicing
start_time = time.time()
for index, row in tqdm(df.iterrows(), total=len(df)):
    question = row['question']
    ref_tables = row['correct_tables']
    selected_table_set = set()
    result_count = 0
    for schema in schema_slicing:
        result_count += 1
        if len(selected_table_set) > 0:
            selected_table =  ', '.join(list(selected_table_set))
        else:
            selected_table = "None"
        instruction = get_instruction(schema)
        input_value = get_input_value(question,selected_table)
        messages = [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{input_value}"}]
        response = predict(messages, model, tokenizer)
        if "#None#" not in response:
            match = re.search(pattern, response)
            if match:
            # 提取匹配的表名字符串
                table_names_str = match.group(1)

                # 使用中文逗号分隔表名，这里我们替换中文逗号为英文逗号并去除空格
        #         table_names = [name.strip() for name in table_names_str.replace('，', ',').split(',')]
                table_names = table_names_str.split(', ')
                # 将table_names整合成一个由英文逗号分隔的字符串
#                 predict_table = ', '.join(table_names)
#                 
#                 print("预测表名：", predict_table)
#                 print("############################################")
                for predict_table in table_names:
                    selected_table_set.add(predict_table)
#                 if result_count == 7 or result_count == 8:
#                     print("^^^^^^^^^^^^^^^^^^^^^final_select_:",predict_table)
            else:
                predict_table = "None"
#                 selected_table_list.append(predict_table)
    if len(selected_table_set) > 0:
        total_selected_table =  ', '.join(list(selected_table_set))
    else:
        total_selected_table = "None"
    print(f"++++++++++++++++index:{index}++++++++++++++++++++++++")
    print("正确表名：",ref_tables)
    print("预测表名：",total_selected_table)
    print("############################################")
    results.append([ref_tables,total_selected_table])
end_time = time.time()
print(len(predict_table))

  1%|▍                                                                                  | 1/178 [00:04<13:28,  4.57s/it]

++++++++++++++++index:0++++++++++++++++++++++++
正确表名： products, manufacturers
预测表名： products, manufacturers
############################################


  1%|▉                                                                                  | 2/178 [00:08<11:57,  4.08s/it]

++++++++++++++++index:1++++++++++++++++++++++++
正确表名： products
预测表名： products
############################################


  2%|█▍                                                                                 | 3/178 [00:12<11:27,  3.93s/it]

...

++++++++++++++++index:175++++++++++++++++++++++++
正确表名： railway
预测表名： railway
############################################


 99%|████████████████████████████████████████████████████████████████████████████████▌| 177/178 [11:28<00:03,  3.88s/it]

++++++++++++++++index:176++++++++++++++++++++++++
正确表名： book
预测表名： book
############################################


100%|█████████████████████████████████████████████████████████████████████████████████| 178/178 [11:31<00:00,  3.89s/it]

++++++++++++++++index:177++++++++++++++++++++++++
正确表名： book, publication
预测表名： publication, book
############################################
4


In [15]:
new_df = pd.DataFrame(results, columns = ['reference_tables','predicted_tables'])

In [16]:
print(new_df)

            reference_tables         predicted_tables
0    products, manufacturers  products, manufacturers
1                   products                 products
2              manufacturers            manufacturers
3              manufacturers                     None
4              manufacturers            manufacturers
..                       ...                      ...
173                     mill                     mill
174                  railway                  railway
175                  railway                  railway
176                     book                     book
177        book, publication        publication, book

[178 rows x 2 columns]


In [17]:
total_data = len(df['question'])
total_time = end_time - start_time
# print(total_data)
signal_infernece_time = total_time/total_data
print(signal_infernece_time)

3.887381667501471


In [18]:
total_samples = len(new_df)
total_accuracy = 0
filtered_accuracy = 0
total_precision = 0
total_recall = 0
count = 0
for index, row in new_df.iterrows():
#     count+=1
#     print("index:",count)
    if not row['predicted_tables'] or pd.isna(row['predicted_tables']):
        continue
    predicted_tables = row['predicted_tables'].split(",")
    reference_tables = row['reference_tables'].split(",")
    
    # Convert to lowercase and strip whitespace for comparison
    predicted_tables = [x.lower().replace("--","").replace("**","").strip() for x in predicted_tables]
    reference_tables = [x.lower().strip() for x in reference_tables]
    
    # Calculate accuracy
    if set(predicted_tables) == set(reference_tables):
        total_accuracy += 1

    # Calculate precision and recall
    true_positives = len(set(predicted_tables) & set(reference_tables))
    false_positives = len(set(predicted_tables) - set(reference_tables))
    false_negatives = len(set(reference_tables) - set(predicted_tables))

    if true_positives == len(reference_tables):
        filtered_accuracy += 1
    
    if len(predicted_tables) > 0:
        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / (true_positives + false_negatives)
    
    total_precision += precision
    total_recall += recall

# Calculate average precision and recall
avg_precision = total_precision / total_samples
avg_recall = total_recall / total_samples

# Calculate total accuracya
accuracy = total_accuracy / total_samples
filtered_accuracy = filtered_accuracy / total_samples

print("Total Accuracy:", accuracy)
print("Filtered Accuracy:", filtered_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)

Total Accuracy: 0.8426966292134831
Filtered Accuracy: 0.8707865168539326
Average Precision: 0.949438202247191
Average Recall: 0.9236891385767791


In [14]:
new_df.to_csv('pre_table.csv',index=False,encoding='utf-8')